In [368]:
import requests, bs4, re, os, time, tqdm, csv, json
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

keyword = "网络个人信息侵权"

# 读取json并转化成DataFrame
file = f"./data/{keyword}_contents.json"
with open(file, "r", encoding="utf-8") as f:
    contents = json.load(f)
df = pd.DataFrame(contents)
# 修改标题
df.rename(columns={"title": "标题", "case_number": "案号", "court": "法院", "data": "日期", "cause": "案由", "type": "类型", "procedure": "程序", "procedure_explain": "程序说明", "tags": "标签", "opinion": "观点", "verdict": "判决"}, inplace=True)


In [ ]:
# 创建一个result文件夹
if not os.path.exists("./result"):
    os.mkdir("./result")

In [369]:
df.head()

,标题,案号,法院,日期,案由,类型,程序,程序说明,标签,观点,判决
0,孙长宝与北京搜狐互联网信息服务有限公司等人格权纠纷一审民事判决书,北京互联网法院,北京互联网法院,2020年09月10日,人格权纠纷,判决书,一审,原告孙某某1与被告北京百度网讯科技有限公司（以下简称百度网讯公司）人格权纠纷一案，本院于20...,"[个人隐私, 网络服务, 追加, 迟延履行, 连带责任, 简易程序, 备案, 免责, 申请人...",本院认为，结合各方当事人的诉辩意见及在案证据，本案的争议焦点为:一、涉案姓名、照片及其关联关...,综上，依照《中华人民共和国民法总则》第一百一十条 、第一百一十一条 ，《中华人民共和国侵权责...
1,吴某、张某、葛某某等侵犯公民个人信息罪一审刑事判决书,滁州市琅琊区人民法院,滁州市琅琊区人民法院,2020年06月03日,侵犯公民个人信息罪,判决书,一审,安徽省滁州市琅琊区人民检察院以琅检一部刑诉[2020]19号起诉书指控被告人吴某某1、张某某...,"[缓刑, 从轻处罚, 取保候审, 并处罚金, 认罪, 如实供述, 证据不足, 自首, 立功,...",本院认为，被告人吴某某1、张某某1、葛某某1、董某某1违反国家规定，非法获取、出售公民个人信...,依照《中华人民共和国刑法》第二百五十三条 之一第一款 、第三款 、第二百八十七条 之一第一款...
2,黄华侵犯公民个人信息罪一审刑事判决书,永泰县人民法院,永泰县人民法院,2021年02月26日,侵犯公民个人信息罪,判决书,一审,永泰县人民检察院以樟检一部刑诉［2021］Z10号起诉书指控被告人黄某某1犯侵犯公民个人信息...,"[缓刑, 从宽处理, 从轻处罚, 取保候审, 并处罚金, 认罪, 如实供述]",本院认为，被告人黄某某1伙同他人违反国家有关规定，向他人出售公民个人信息，情节严重，其行为已...,综上，依照《中华人民共和国刑法》第二百五十三条 之一第一款 、第二十五条 第一款 、第六十一...
3,安思洁、刘平兴隐私权纠纷再审审查与审判监督民事裁定书,四川省高级人民法院,四川省高级人民法院,2019年03月08日,隐私权纠纷,裁定书,再审,再审申请人安某某1因与被申请人刘某某1、王某某1隐私权纠纷一案，不服四川省绵阳市中级人民法院...,,,依照《中华人民共和国民事诉讼法》第二百零四条 第一款 ，《最高人民法院关于适用的解释》第三百...
4,苏州贝尔塔数据技术有限公司与伊日克斯庆一般人格权纠纷二审民事判决书,江苏省苏州市中级人民法院,江苏省苏州市中级人民法院,2020年06月10日,一般人格权纠纷,判决书,二审,上诉人苏州贝尔塔数据技术有限公司（以下简称贝尔塔公司）因与被上诉人伊某某1人格权纠纷一案，不...,"[备案, 网络服务, 工商登记, 转载, 新证据, 公证书, 法定程序, 迟延履行, 加倍支...",本院认为，根据《中华人民共和国民法总则》第一百一十一条 之规定，自然人的个人信息受法律保护。...,本院依照《中华人民共和国民法总则》第一百一十一条 、《中华人民共和国侵权责任法》第二十条 、...


In [370]:
# 转换日期格式 2020年06月10日 -> 2020-06-10
df["日期"] = df["日期"].apply(lambda x: x.replace("年", "-").replace("月", "-").replace("日", ""))
# 改变日期数据类型
df["日期"] = pd.to_datetime(df["日期"])
# 查看数据类型
df.dtypes

标题              object
案号              object
法院              object
日期      datetime64[ns]
案由              object
类型              object
程序              object
程序说明            object
标签              object
观点              object
判决              object
dtype: object

In [371]:
len(df)

400

In [372]:
# 获取['判决']中所有被《》包裹的内容，放进一个数组
laws = df["判决"].apply(lambda x: re.findall(r"《(.*?)》", x))
# 合并law里面的所有，去掉重复项
laws = list(set([i for j in laws for i in j]))
# 只留下laws中带有“法”字的
laws = [i for i in laws if "法" in i]
# 查看laws
for laws in laws:
    print(laws)

中华人民共和国合同法
最高人民法院、最高人民检察院《关于办理侵犯公民个人信息刑事案件适用法律若干问题的解释
最高人民法院关于适用《中华人民共和国民事诉讼法
最高人民法院《关于审理利用信息网络侵害人身权益民事纠纷案件适用法律若干问题的规定
中华人民共和国民法总则
中国人民共和国民法总则
中华人民共和国继承法
最高人民法院关于民事诉讼证据若干问题的规定
最高人民法院、最高人民检察院关于检察公益诉讼案件适用法律若干问题的解释
关于办理侵犯公民个人信息刑事案件适用法律若干问题的解释
中华人民共和国网络安全法
最高人民法院关于审理利用信息网络侵害人身权益民事案件适用法律若干问题的规定
中华人民共和国民法通则
中华人民共和国刑法
最高人民法院关于处理自首和立功具体应用法律若干问题的解释
最高人民法院关于适用《中华人民共和国刑事诉讼法
中华人民共和国刑法修正案（八）
最高人民法院关于确定民事侵权精神损害陪若干问题的解释
最高人民法院关于审理利用信息网络侵害人身权益民事纠纷案件适用法律若干问题的解释
中华人民共和国公司法
中华人民共和国民法典
最高人民法院关于审理不正当竞争民事案件应用法律若干问题的解释
最高人民法院关于审理利用信息网络侵害人身权益民事纠纷案件适用法律问题的规定
最高人民法院关于审理利用信息网络侵害人身权益民事纠纷案件适用法律若干问题的的规定
最高人民法院关于审理侵害信息网络传播权民事纠纷案件适用法律若干问题的规定
中华人民共和国著作权法
关于审理侵害信息网络传播权民事纠纷案件适用法律若干问题的规定
中华人民共和国刑事诉讼法
最高人民法院关于民事诉讼证据的若干规定
最高人民法院关于审理利用信息网络侵害民事权益纠纷案件适用法律若干问题的规定
最高人民法院关于适用的解释
最高人民法院、最高人民检察院
中国人民共和国民事诉讼法
中华人民共和国侵权法
中华人民共和国涉外民事关系法律适用法
中华人民共和国著作权法实施条例
最高人民法院关于贯彻执行《中华人民共和国民法通则
中华人民共和民法通则
最高人民法院关于审理利用信息网络侵害人身权益民事纠纷案件适用法律若干问题的规定
最高人民法院关于适用《中华人民共和国民法典
中国人民共和国侵权责任法
最高人民法院关于审理著作权民事纠纷案件适用法律若干问题的解释
最高人民法院关于审理名誉权案件若干问题的解释
中华人民共和国侵权责任法
最高

In [373]:
# 读取 law_dic.txt 里面存放着所有的法律文书名称，我要用来做词云
with open("./law_dic.txt", "r", encoding="utf-8") as f:
    law_dic = f.read().split("\n")
# 查看law_dic
for law in law_dic:
    print(law)

中华人民共和国合同法
关于办理侵犯公民个人信息刑事案件适用法律若干问题的解释
中华人民共和国民事诉讼法
中华人民共和国民法总则
关于确定民事侵权精神损害赔偿责任若干问题的解释
中华人民共和国民法典
关于检察公益诉讼案件适用法律若干问题的解释
关于审理消费民事公益诉讼适用法律若干问题的解释
中华人民共和国网络安全法
关于适用时间效力的若干规定
中华人民共和国民法通则
中华人民共和国消费者权益保护法
关于审理利用信息网络侵害人身权益民事纠纷案件适用法律若干问题的规定
中华人民共和国民法典
关于办理侵犯公民个人信息刑事案件适用法律若干问题的解释
中华人民共和国刑法
中华人民共和国民事诉讼法
中华人民共和国侵权责任法
中华人民共和国刑事诉讼法
中华人民共和国刑事诉讼法
关于民事诉讼证据的若干规定


In [374]:
# 统计所有 '判决'中 law_dic 里面的词出现的次数，然后做词云
# 用一个字典来存放所有的词和出现的次数
law_dic_count = {}
# 遍历所有的词
for law in law_dic:
    # 统计每个词出现的次数
    law_dic_count[law] = df["判决"].apply(lambda x: x.count(law)).sum()
# 查看law_dic_count
for k, v in law_dic_count.items():
    print(k, v)
# 生成词云
wc = WordCloud(font_path="./fonts/微软雅黑.ttf", background_color="white", width=1000, height=800, max_words=100, max_font_size=150)
wc.generate_from_frequencies(law_dic_count)
plt.figure(figsize=(10, 5), dpi=300)
# 保存
wc.to_file(f"./result/{keyword}_法条词云.png")

# 先对law_dic_count进行排序
law_dic_count = dict(sorted(law_dic_count.items(), key=lambda x: x[1], reverse=False)[:12])
# 对law_dic_count绘制横向的bar图
plt.figure(figsize=(10, 7), dpi=300)
plt.xlabel("出现次数")
plt.ylabel("法条")
plt.title(f"{keyword}_法条词频")
plt.barh(list(law_dic_count.keys()), list(law_dic_count.values()))
plt.savefig(f"./result/{keyword}_法条词频.png", dpi=300)


中华人民共和国合同法 3
关于办理侵犯公民个人信息刑事案件适用法律若干问题的解释 22
中华人民共和国民事诉讼法 358
中华人民共和国民法总则 91
关于确定民事侵权精神损害赔偿责任若干问题的解释 51
中华人民共和国民法典 20
关于检察公益诉讼案件适用法律若干问题的解释 26
关于审理消费民事公益诉讼适用法律若干问题的解释 0
中华人民共和国网络安全法 23
关于适用时间效力的若干规定 7
中华人民共和国民法通则 43
中华人民共和国消费者权益保护法 12
关于审理利用信息网络侵害人身权益民事纠纷案件适用法律若干问题的规定 61
中华人民共和国刑法 62
中华人民共和国侵权责任法 178
中华人民共和国刑事诉讼法 44
关于民事诉讼证据的若干规定 16


In [375]:
# 对'案由'做词频统计，做词云
# 用一个字典来存放所有的词和出现的次数
cause_count = {}
# 遍历所有的词
for cause in df["案由"]:
    # 统计每个词出现的次数
    cause_count[cause] = df["案由"].apply(lambda x: x.count(cause)).sum()
# 查看cause_count
for k, v in cause_count.items():
    print(k, v)
# 生成词云
wc = WordCloud(font_path="./fonts/微软雅黑.ttf", background_color="white", width=1000, height=800, max_words=100, max_font_size=150)
wc.generate_from_frequencies(cause_count)
plt.figure(figsize=(10, 5), dpi=300)
# 保存
wc.to_file(f"./result/{keyword}_案由词云.png")

人格权纠纷 44
侵犯公民个人信息罪 45
隐私权纠纷 62
一般人格权纠纷 28
网络侵权责任纠纷 76
侵权责任纠纷 102
民事案由 21
商业贿赂不正当竞争纠纷 1
名誉权纠纷 72
不正当竞争纠纷 4
肖像权纠纷 5
姓名权纠纷 6
人事争议 1
侵害作品信息网络传播权纠纷 11
网络域名权属纠纷 1
航空旅客运输合同纠纷 3
网络购物合同纠纷 1
财产损害赔偿纠纷 3
其他行政管理 1
敲诈勒索罪 1
著作权权属、侵权纠纷 3
非法利用信息网络罪 1
合同纠纷 8
借记卡纠纷 1
非法获取公民个人信息罪 2
道路交通管理（道路） 1
买卖合同纠纷 1
劳动争议 1
信用卡纠纷 1
网络服务合同纠纷 1
行政复议 2
侵害作品复制权纠纷 1


In [376]:
# 对案由绘制横向的bar图，横坐标为数量，纵坐标为案由
# 先排序
cause_count = dict(sorted(cause_count.items(), key=lambda x: x[1], reverse=True)[:10])
sns.barplot(x=list(cause_count.values()), y=list(cause_count.keys()))
title = f"{keyword}_案由词频"
plt.xlabel("数量")
plt.ylabel("案由")
plt.title(title)
# 保存
plt.savefig(f"./result/{keyword}_案由词频.png", dpi=300)


In [377]:
# 连接所有标签，做词频统计，保存词云
tags = []
for tag in df["标签"]:
    tags.extend(tag)
# 统计每个词出现的次数
tags_count = {}
for tag in tags:
    tags_count[tag] = tags.count(tag)
# 查看tags_count
for k, v in tags_count.items():
    print(k, v)
# 生成词云
wc = WordCloud(font_path="./fonts/微软雅黑.ttf", background_color="white", width=1000, height=800, max_words=100, max_font_size=150)
wc.generate_from_frequencies(tags_count)
plt.figure(figsize=(10, 5), dpi=300)
# 保存
wc.to_file(f"./result/{keyword}_标签词云.png")

# 对标签根据数量进行排序，绘制横向的bar图，横坐标为数量，纵坐标为标签，只绘制排名前10的
tags_count = sorted(tags_count.items(), key=lambda x: x[1], reverse=True)[:10]
tags_count = dict(tags_count)
sns.barplot(x=list(tags_count.values()), y=list(tags_count.keys()))
title = f"{keyword}_标签词频"
plt.xlabel("数量")
plt.ylabel("标签")
plt.title(title)
# 保存
plt.savefig(f"./result/{keyword}_标签词频.png", dpi=300)



个人隐私 132
网络服务 157
追加 12
迟延履行 98
连带责任 85
简易程序 124
备案 24
免责 12
申请人 24
法定义务 19
证据保全 33
公证书 103
转为普通程序 5
名誉 145
票据 5
注意义务 43
普通程序 53
投诉 79
关联性 50
代理 249
救济 15
违反法律 40
抗辩 49
加倍支付 102
委托 245
缓刑 23
从轻处罚 42
取保候审 27
并处罚金 44
认罪 39
如实供述 36
证据不足 65
自首 9
立功 8
情节特别严重 30
从宽处理 9
工商登记 5
转载 49
新证据 35
法定程序 6
审判委员会 1
损害赔偿 116
损害事实 33
书面方式 5
网络侵权 103
知情权 14
当庭自愿认罪 5
查阅 10
复议 4
恶意诉讼 3
撤回上诉 30
监护人 4
精神损害 177
收购 4
共同侵权 11
监管 27
破产 3
物流 10
注销 22
破产清算 2
对账 3
营业执照 13
担保 8
告知义务 9
现场勘验 1
司法鉴定 19
延期审理 11
抗诉 4
减轻处罚 11
传唤 25
评估报告 2
网络游戏 4
依据不足 16
回收 1
公共利益 50
程序违法 11
不动产 1
劳动合同 9
社会保险 2
解除 12
投资 16
劳动关系 6
无效 6
劳动争议 8
退赃 9
初犯 5
利润 4
年费 1
专利 11
反诉 10
安全保障 11
注册资金 1
行政处罚 17
逾期 29
询问笔录 11
适格 38
物业管理 7
主体资格 14
中止审理 5
微信公众号 43
合并审理 11
管辖 2
设备所在地 5
国家秘密 6
拘留 4
民间借贷 5
恶意串通 1
汇款凭证 1
不开庭审理 11
商业秘密 7
怠于行使 1
适用法律错误 3
保险公司 4
通知义务 4
连带赔偿 10
安全标准 3
聘用 5
工资表 1
补充协议 3
赠送 8
恢复原状 11
分割 2
积极退赃 2
专属管辖 3
管辖权异议 8
合同履行地 3
书面协议 3
撤销权 1
违约金 5
辞职 8
管理人 15
变更登记 2
公司设立 4
付清 3
安全防护 6
检验报告 2
保证金 2
租赁合同 3
强制执行 2
租赁 8
淘宝网店 2
另行起诉 6
法定代表人 21
筹备 6
辞退 

In [378]:
# 使中文字体正常显示，用微软雅黑
plt.rcParams["font.sans-serif"] = ["Microsoft YaHei"]

# 对df通过日期排序
df.sort_values(by="日期", inplace=True)
# 统计每个年份的案件数量
year_count = df["日期"].apply(lambda x: x.year).value_counts()
# 查看year_count
for k, v in year_count.items():
    print(k, v)
# 绘制柱状图，并用一条曲线拟合,用sns
plt.figure(figsize=(10, 7), dpi=300)
sns.barplot(x=year_count.index, y=year_count.values)
plt.xticks(rotation=45)
plt.xlabel("年份")
plt.ylabel("案件数量")
plt.title(f"{keyword}案件数量随时间变化")
# 保存
plt.savefig(f"./result/{keyword}_案件数量随时间变化.png")

2020.0 110
2019.0 83
2021.0 58
2018.0 46
2015.0 31
2017.0 24
2016.0 23
2014.0 8
2012.0 5
2010.0 3
2013.0 3
2008.0 2
2011.0 2
2009.0 1


In [379]:
# 提取'法院'里面的省份
provinces = df["法院"].apply(lambda x: x[:2])
# provinces转成一个数组
provinces = provinces.values
len(provinces)

400

In [387]:
province_city_dic = {
    '北京': ['北京'],
    '上海': ['上海'],
    '深圳': ['深圳'],
    '天津': ['天津'],
    '重庆': ['重庆'],
    '澳门': ['澳门'],
    '香港': ['香港'],
    '海南': ['海口', '三亚'],
    '台湾': ['台湾', '台北', '高雄', '基隆', '台中', '台南', '新竹', '嘉义'],
    '河北': ['唐山', '邯郸', '邢台', '保定', '承德', '沧州', '廊坊', '衡水', '石家庄', '秦皇岛', '张家口'],
    '山西': ['太原', '大同', '阳泉', '长治', '晋城', '朔州', '晋中', '运城', '忻州', '临汾', '吕梁'],
    '山东': ['济南', '青岛', '淄博', '枣庄', '东营', '烟台', '潍坊', '济宁', '泰安', '威海', '日照', '莱芜', '临沂', '德州', '聊城', '滨州', '荷泽', '菏泽'],
    '江苏': ['南京', '无锡', '徐州', '常州', '苏州', '南通', '淮安', '盐城', '扬州', '镇江', '泰州', '宿迁', '连云港'],
    '浙江': ['杭州', '宁波', '温州', '嘉兴', '湖州', '绍兴', '金华', '衢州', '舟山', '台州', '丽水'],
    '安徽': ['合肥', '芜湖', '蚌埠', '淮南', '淮北', '铜陵', '安庆', '黄山', '滁州', '阜阳', '宿州', '巢湖', '六安', '亳州', '池州', '宣城', '马鞍山'],
    '福建': ['福州', '厦门', '莆田', '三明', '泉州', '漳州', '南平', '龙岩', '宁德'],
    '江西': ['南昌', '萍乡', '新余', '九江', '鹰潭', '赣州', '吉安', '宜春', '抚州', '上饶', '景德镇'],
    '河南': ['郑州', '开封', '洛阳', '焦作', '鹤壁', '新乡', '安阳', '濮阳', '许昌', '漯河', '南阳', '商丘', '信阳', '周口', '驻马店', '济源', '平顶山', '三门峡'],
    '湖北': ['武汉', '黄石', '襄樊', '十堰', '荆州', '宜昌', '荆门', '鄂州', '孝感', '黄冈', '咸宁', '随州', '恩施', '仙桃', '天门', '潜江'],
    '湖南': ['长沙', '株洲', '湘潭', '衡阳', '邵阳', '岳阳', '常德', '益阳', '郴州', '永州', '怀化', '娄底', '吉首', '张家界'],
    '广东': ['广州', '深圳', '珠海', '汕头', '韶关', '佛山', '江门', '湛江', '茂名', '肇庆', '惠州', '梅州', '汕尾', '河源', '阳江', '清远', '东莞', '中山', '潮州', '揭阳', '云浮'],
    '广西': ['南宁', '柳州', '桂林', '梧州', '北海', '钦州', '贵港', '玉林', '百色', '贺州', '河池', '来宾', '崇左', '防城港'],
    '四川': ['成都', '自贡', '泸州', '德阳', '绵阳', '广元', '遂宁', '内江', '乐山', '南充', '宜宾', '广安', '达州', '眉山', '雅安', '巴中', '资阳', '西昌', '攀枝花'],
    '贵州': ['贵阳', '遵义', '安顺', '铜仁', '毕节', '兴义', '凯里', '都匀', '六盘水', '黔西南布依族苗族自治州', '黔东南苗族侗族自治州', '黔南布依族苗族自治州'],
    '云南': ['昆明', '曲靖', '玉溪', '保山', '昭通', '丽江', '思茅', '临沧', '景洪', '楚雄', '大理', '潞西'],
    '陕西': ['西安', '铜川', '宝鸡', '咸阳', '渭南', '延安', '汉中', '榆林', '安康', '商洛'],
    '甘肃': ['兰州', '金昌', '白银', '天水', '武威', '张掖', '平凉', '酒泉', '庆阳', '定西', '陇南', '临夏', '合作', '嘉峪关'],
    '辽宁': ['沈阳', '大连', '鞍山', '抚顺', '本溪', '丹东', '锦州', '营口', '盘锦', '阜新', '辽阳', '铁岭', '朝阳', '葫芦岛'],
    '吉林': ['长春', '吉林', '四平', '辽源', '通化', '白山', '松原', '白城', '延吉'],
    '黑龙江': ['鹤岗', '鸡西', '大庆', '伊春', '黑河', '绥化', '双鸭山', '牡丹江', '佳木斯', '七台河', '哈尔滨', '齐齐哈尔'],
    '青海': ['西宁', '德令哈', '格尔木'],
    '宁夏': ['银川', '吴忠', '固原', '中卫', '石嘴山'],
    '西藏': ['拉萨', '日喀则'],
    '新疆': ['哈密', '和田', '喀什', '昌吉', '博乐', '伊宁', '塔城', '吐鲁番', '阿图什', '库尔勒', '五家渠', '阿克苏', '阿勒泰', '石河子', '阿拉尔', '乌鲁木齐', '克拉玛依', '图木舒克'],
    '内蒙古': ['包头', '乌海', '赤峰', '通辽', '鄂尔多斯', '呼伦贝尔', '巴彦淖尔', '乌兰察布', '兴安盟', '呼和浩特', '锡林郭勒盟', '阿拉善盟', '巴彦淖尔盟', '乌兰察布盟']
}
# 根据province_city_dic,把province里面的一些城市改成对应的省份
for idx, pro in enumerate(provinces):
    for pro_, cities in province_city_dic.items():
        if pro in cities:
            provinces[idx] = pro_
            # print(f"城市{pro}在{pro_}省")
            break

print(provinces)

['北京' '北京' '上海' '上海' '上海' '浙江' '上海' '北京' '上海' '河南' '浙江' '陕西' '上海' '江苏'
 '上海' '北京' '北京' '上海' '石家' '天津' '连云' '河北' '广东' '江苏' '江苏' '湖南' '乐清' '辽宁'
 '江苏' '江苏' '吉林' '江苏' '定远' '定远' '定远' '广东' '浙江' '滕州' '滕州' '江苏' '深圳' '云南'
 '镇原' '镇原' '镇原' '上海' '北京' '北京' '吉林' '江苏' '山东' '广东' '北京' '北京' '上海' '安徽'
 '上海' '云南' '海南' '北京' '吉林' '江西' '北京' '江苏' '江苏' '四川' '广东' '广东' '广东' '辽宁'
 '广东' '吉林' '上海' '陕西' '广东' '广东' '浙江' '望奎' '如东' '深圳' '浙江' '北京' '广东' '梁山'
 '江苏' '福建' '湖北' '浙江' '广东' '泗洪' '呼和' '呼和' '北京' '湖北' '江苏' '上海' '江苏' '上海'
 '湖南' '四川' '辽宁' '北京' '江苏' '上海' '湖北' '江苏' '安徽' '辽宁' '北京' '海南' '湖南' '辽宁'
 '广东' '安徽' '深圳' '四川' '辽宁' '陕西' '闽侯' '吉林' '四川' '辽宁' '北京' '瓦房' '浙江' '四川'
 '北京' '上海' '浙江' '广东' '武胜' '重庆' '江苏' '靖江' '北京' '四川' '神木' '神木' '神木' '神木'
 '北京' '北京' '山东' '浙江' '山东' '江苏' '广西' '北京' '辽宁' '安徽' '四川' '浙江' '北京' '北京'
 '北京' '天津' '苍南' '广东' '广东' '广东' '重庆' '北京' '安徽' '陕西' '北京' '上海' '上海' '北京'
 '北京' '广东' '石家' '北京' '广东' '浙江' '山东' '浙江' '北京' '北京' '黑龙' '广西' '广西' '广西'
 '广西' '北京' '黑龙' '山东' '安徽' '上海' '上海' '上海' '上海' '上海' '上海' '上海' '上海' '上海'
 '上海' 

In [390]:
# 对 provinces做词频统计，并在地图上标注出数量
provinces_count = {}
for pro in provinces:
    if pro not in provinces_count:
        provinces_count[pro] = 0
    provinces_count[pro] += 1



In [ ]:
! pip install echarts-countries-pypkg #世界地图
! pip install echarts-china-provinces-pypkg #中国省份
! pip install echarts-china-cities-pypkg #中国城市
! pip install echarts-china-counties-pypkg #中国县区
! pip install echarts-china-misc-pypkg #中国区域